In [42]:
import submission
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

## Parameters settings
past_cases_interval = 10
past_weather_interval = 10


## Read training data
train_file = './data/COVID_train_data.csv'
train_df = pd.read_csv(train_file)

## Read Training labels
train_label_file = './data/COVID_train_labels.csv'
train_labels_df = pd.read_csv(train_label_file)


## Read testing Features
test_fea_file = './data/test_features.csv'
test_features = pd.read_csv(test_fea_file)


## Set hyper-parameters for the SVM Model
svm_model = SVR()
svm_model.set_params(**{'kernel': 'rbf', 'degree': 1, 'C': 5000,
                        'gamma': 'scale', 'coef0': 0.0, 'tol': 0.001, 'epsilon': 10})

SVR(C=5000, degree=1, epsilon=10)

In [89]:
## Project-Part1
import math
def predict_COVID_part1(svm_model, train_df, train_labels_df, past_cases_interval, past_weather_interval, test_feature):
    #处理 train_df
    # [max_temp, max_dew, max_humid, past_cases]

    df3 = pd.DataFrame(columns = ['day'], data= [x for x in range(31,193)])
    col_names = ["max_temp","max_dew","max_humid"]
    # 循环列名 i
    for i in range(len(col_names)):
        # 循环天数 j
        for j in range(past_weather_interval,0, -1):
            col_name = col_names[i]+"-"+str(j)
            df3[col_name] = np.nan
            # 循环数据 k
            for k in range(0,162):
                df3.loc[k,col_name] = train_df.loc[df3.loc[k,'day']-j-1, col_names[i]]

    for j in range(past_cases_interval,0, -1):
        col_name = "dailly_cases"+"-"+str(j)
        df3[col_name] = np.nan
        # 循环数据 k
        for k in range(0,162):
            df3.loc[k,col_name] = train_df.loc[df3.loc[k,'day']-j-1, 'dailly_cases']
    
    #训练模型
    train_data = np.array(df3.drop(["day"], axis=1))
    train_label = train_labels_df.loc[30:191,'dailly_cases'].values

    svm_model.fit(train_data, train_label)
    
    df3_col_list = df3.columns.tolist()
    test_feature_col_list = test_feature.index.tolist()
    diff_col = list(set(test_feature_col_list).difference(set(df3_col_list)))
    test_data = test_feature.drop(index = diff_col, axis = 1)
    test_data = np.array([np.array(test_data.drop(["day"]))])
    
    return math.floor(svm_model.predict(test_data))

In [90]:
predicted_cases_part1 = []
for idx in range(len(test_features)):
    test_feature = test_features.loc[idx]
    prediction = predict_COVID_part1(svm_model, train_df, train_labels_df, 
                                                past_cases_interval, past_weather_interval, test_feature)
    predicted_cases_part1.append(prediction)

In [91]:
print(predicted_cases_part1)

[945, 897, 832, 881, 907, 921, 1028, 819, 812, 809, 860, 845, 837, 898, 861, 811, 846, 839, 855, 892]


In [92]:
y = [945, 897, 832, 881, 907, 921, 1028, 819, 812, 809, 860, 845, 837, 898, 861, 811, 846, 839, 855, 892]
diff = []
for i in range(len(y)):
    diff.append(y[i]-predicted_cases_part1[i])
print(diff)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
